# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [33]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import requests
import json
import numpy as np

# Import API key
from api_keys import geoapify_key

In [34]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Sharlyk,52.9167,54.7500,8.64,82,7,3.36,RU,1714589042
1,1,Waitangi,-43.9535,-176.5597,12.68,97,100,3.13,NZ,1714589043
2,2,Mount Isa,-20.7333,139.5000,20.87,56,40,5.66,AU,1714589043
3,3,West Island,-12.1568,96.8225,27.99,94,75,3.60,CC,1714589044
4,4,Punta Arenas,-53.1500,-70.9167,5.06,70,75,3.09,CL,1714588848


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [35]:
%%capture --no-display

# Configure the map plot
df = city_data_df[["City","Lat","Lng","Humidity"]]
hum =city_data_df["Humidity"]
city = city_data_df['City']

# Display the map
df = gpd.GeoDataFrame(df, geometry= gpd.points_from_xy(df.Lng,df.Lat))

geo_plot =df.hvplot("Lng",
                    "Lat",
                    color = "City",
                    geo = True,
                    tiles = True,
                    size = hum,
                    width = 1200,
                    height = 800)

geo_plot



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,_size)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [36]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_criteria = city_data_df[(city_data_df['Max Temp']<27)& \
                                  (city_data_df['Max Temp']>21)& \
                                  (city_data_df['Wind Speed']<4.5)& \
                                  (city_data_df['Cloudiness']==0)]

# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
city_data_criteria

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,29,Holualoa,19.6228,-155.9522,25.48,74,0,1.54,US,1714589066
45,45,Tôlanaro,-25.0319,46.9987,24.04,83,0,1.03,MG,1714589081
89,89,Tsiombe,-25.3000,45.4833,21.59,90,0,0.97,MG,1714589119
94,94,Mhamid,29.8200,-5.7200,26.93,10,0,4.48,MA,1714589124
176,176,El Granada,37.5027,-122.4694,24.38,56,0,2.57,US,1714589199
234,234,Pacific Grove,36.6177,-121.9166,23.03,68,0,4.47,US,1714589249
237,237,Toliara,-23.3500,43.6667,22.54,83,0,3.71,MG,1714589252
269,269,Cologne,50.9333,6.9500,23.79,60,0,2.57,DE,1714589158
401,401,Mahendragarh,28.2833,76.1500,22.96,20,0,3.03,IN,1714589397
416,416,Port Hedland,-20.3167,118.5667,21.03,24,0,3.09,AU,1714589410


### Step 3: Create a new DataFrame called `hotel_df`.

In [37]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_criteria.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel_Name"]=''

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel_Name
29,29,Holualoa,19.6228,-155.9522,25.48,74,0,1.54,US,1714589066,
45,45,Tôlanaro,-25.0319,46.9987,24.04,83,0,1.03,MG,1714589081,
89,89,Tsiombe,-25.3000,45.4833,21.59,90,0,0.97,MG,1714589119,
94,94,Mhamid,29.8200,-5.7200,26.93,10,0,4.48,MA,1714589124,
176,176,El Granada,37.5027,-122.4694,24.38,56,0,2.57,US,1714589199,
234,234,Pacific Grove,36.6177,-121.9166,23.03,68,0,4.47,US,1714589249,
237,237,Toliara,-23.3500,43.6667,22.54,83,0,3.71,MG,1714589252,
269,269,Cologne,50.9333,6.9500,23.79,60,0,2.57,DE,1714589158,
401,401,Mahendragarh,28.2833,76.1500,22.96,20,0,3.03,IN,1714589397,
416,416,Port Hedland,-20.3167,118.5667,21.03,24,0,3.09,AU,1714589410,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [38]:
# https://api.geoapify.com/v2/places?
# categories=public_transport.subway
# &filter=circle:-0.07071648508463113,51.50848194136378,1000
# &bias=proximity:-0.07071648508463113,51.50848194136378
# &limit=20&apiKey=YOUR_API_KEY

base_url1 = "https://api.geoapify.com/v2/places"
categories = "accommodation.hotel"
lon =  "-155.9522"
lat =	"19.6228"
raduis = 10000
filters = f"circle:{lon},{lat},{raduis}"
bias = f"proximity:{lon},{lat}"
limit = 5


In [39]:
params = {"categories": categories,
          "filters": filters,
          "bias": bias,
          "limit": limit,
          "apiKey":geoapify_key          
}

In [40]:
response2 = requests.get(base_url1, params=params).json()

In [41]:
response2['features'][0]['properties']['name']

'Kona Hotel'

In [42]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
raduis = 10000
filters = f"circle:{lon},{lat},{raduis}"
bias = f"proximity:{lon},{lat}"
limit = 5

params = {"categories": categories,
          "filters": filters,
          "bias": bias,
          "limit": limit,
          "apiKey":geoapify_key          
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lon = row['Lng']

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{raduis}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url1, params=params)
    
    # Convert the API response to JSON format
    name_address = response2 = requests.get(base_url1, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Holualoa - nearest hotel: Kona Hotel
Tôlanaro - nearest hotel: Hôtel Mahavokey
Tsiombe - nearest hotel: No hotel found
Mhamid - nearest hotel: Hotel Kasbah Azalay
El Granada - nearest hotel: Beach House
Pacific Grove - nearest hotel: Pacific Grove Inn
Toliara - nearest hotel: Ambary
Cologne - nearest hotel: Wasserturm Hotel Cologne
Mahendragarh - nearest hotel: No hotel found
Port Hedland - nearest hotel: The Esplanade Hotel
Tura - nearest hotel: Hotel Polo Orchid
Cambria - nearest hotel: Olallieberry Inn
Takhatgarh - nearest hotel: No hotel found
Macon - nearest hotel: Decatur Inn
Arica - nearest hotel: Hotel Andalucía 


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel_Name,Hotel Name
29,29,Holualoa,19.6228,-155.9522,25.48,74,0,1.54,US,1714589066,,Kona Hotel
45,45,Tôlanaro,-25.0319,46.9987,24.04,83,0,1.03,MG,1714589081,,Hôtel Mahavokey
89,89,Tsiombe,-25.3000,45.4833,21.59,90,0,0.97,MG,1714589119,,No hotel found
94,94,Mhamid,29.8200,-5.7200,26.93,10,0,4.48,MA,1714589124,,Hotel Kasbah Azalay
176,176,El Granada,37.5027,-122.4694,24.38,56,0,2.57,US,1714589199,,Beach House
234,234,Pacific Grove,36.6177,-121.9166,23.03,68,0,4.47,US,1714589249,,Pacific Grove Inn
237,237,Toliara,-23.3500,43.6667,22.54,83,0,3.71,MG,1714589252,,Ambary
269,269,Cologne,50.9333,6.9500,23.79,60,0,2.57,DE,1714589158,,Wasserturm Hotel Cologne
401,401,Mahendragarh,28.2833,76.1500,22.96,20,0,3.03,IN,1714589397,,No hotel found
416,416,Port Hedland,-20.3167,118.5667,21.03,24,0,3.09,AU,1714589410,,The Esplanade Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [55]:
%%capture --no-display

# Configure the map plot

df = hotel_df[["City","Lat","Lng","Humidity","Country", "Hotel Name"]]
# hum = hotel_df["Humidity"]
# city = hotel_df['City']

# Display the map
df = gpd.GeoDataFrame(df, geometry= gpd.points_from_xy(df.Lng,df.Lat))

geo_plot = df.hvplot.points("Lng",
                            "Lat",
                            color = "City",
                            geo = True,
                            tiles = True,
                            size = "Humidity",
                            width = 1200,
                            height = 800,
                            hover_cols = ["Country", "Hotel Name"]
                            )

# Display the map
geo_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)